In [1]:
import numpy as np
import torch

In [2]:
from sklearn.manifold import TSNE
from sklearn.manifold import Isomap

In [3]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# https://plotly.com/python/creating-and-updating-figures/
import plotly.graph_objects as go

In [4]:
import sys
sys.path.append('../') # now can access entire repository, (important when running locally)

# from orbit_datasets import neworbits, versatileorbits, staticorbits
from ldcl.data import neworbits, versatileorbits, staticorbits

# Data Generation

In [5]:
orbits_dataset = neworbits.OrbitsDataset()
orbits_loader = torch.utils.data.DataLoader(
    dataset = orbits_dataset,
    shuffle = True,
    batch_size = 1,
)

It took 3.58799409866333 time to finish the job.


In [6]:
def getRepresentations(encoder_location):
    branch_encoder = torch.load(encoder_location, map_location=torch.device('cpu'))
    branch_encoder.eval()
    
    encoder_outputs_list = []
    target_values = []

    for it, (input1, input2, y) in enumerate(orbits_loader):
        predicted_representation = branch_encoder(input1.float()).detach().numpy()[0]
        encoder_outputs_list.append(predicted_representation)

        #append conserved quantities to the end of the representation for plotting, y = (1, )
        #[2=phi0,3=energy,4=angular_momentum] discard [0=eccentricity, 1=semimajor_axis]
        target_values.append(np.array([y[0][2],y[0][3],y[0][4]]))
        
    encoder_outputs = np.vstack(encoder_outputs_list)
    target_values = np.vstack(target_values)

    phi0_c_values = target_values[:,0]
    energy_c_values = target_values[:,1]
    angular_momentum_c_values = target_values[:,2]
    
    return encoder_outputs, [phi0_c_values, energy_c_values, angular_momentum_c_values]

# Visualizing Models

In [7]:
# max is 10240
NUM_SHOWN = 10000

In [8]:
ENCODER_PATH = "saved_models/3Dorbits_rmseNCE/final_encoder.pt"

# ENCODER_PATH = "saved_models/3Dorbits_mseNCE/40_encoder.pt"
# ENCODER_PATH = "saved_models/3Dorbits_mseNCE/80_encoder.pt"

# ENCODER_PATH = "saved_models/4Dorbits_rmseNCE/final_encoder.pt"
# ENCODER_PATH = "saved_models/4Dorbits_mseNCE/40_encoder.pt"

In [9]:
encoder_outputs, conserved_quantities = getRepresentations(ENCODER_PATH)

In [10]:
dims = [encoder_outputs[:NUM_SHOWN,i] for i in range(encoder_outputs.shape[1])]
phi0_c_values, energy_c_values, angular_momentum_c_values = conserved_quantities
conserved_quantities = [phi0_c_values[:NUM_SHOWN], energy_c_values[:NUM_SHOWN], angular_momentum_c_values[:NUM_SHOWN]]

In [11]:
# conserved_quantities = [phi0_c_values, energy_c_values, angular_momentum_c_values]
color = conserved_quantities[1]

fig = go.Figure(
        data=[go.Scatter3d(
        x=dims[0],
        y=dims[1],
        z=dims[2],
        mode='markers',
        marker=dict(
            size=4,
            color=color, # set color to an array/list of desired values
            colorscale='viridis',
            opacity=1
        )
    )],
    layout=go.Layout(height=800, width=1000)
)
fig.show()

### Dimensional Reduction with t-SNE

We also try t-SNE. There only seems to be one manifold so it's not really playing to t-SNE's strengths in this case but I'm saving everything in case it's useful.

In [12]:
dr = TSNE(n_components=3, init = 'random',learning_rate = 'auto', perplexity=20)

projected_outputs = dr.fit_transform(encoder_outputs) # projected_outputs.shape = (10240,3)

In [ ]:
dims = [projected_outputs_3d[:NUM_SHOWN,i] for i in range(projected_outputs_3d.shape[1])]

In [29]:
# conserved_quantities = [phi0_c_values, energy_c_values, angular_momentum_c_values]
color = conserved_quantities[0]

fig = go.Figure(
        data=[go.Scatter3d(
        x=dims[0],
        y=dims[1],
        z=dims[2],
        mode='markers',
        marker=dict(
            size=4,
            color=color, # set color to an array/list of desired values
            colorscale='viridis',
            opacity=1
        )
    )],
    layout=go.Layout(height=800, width=1000)
)
fig.show()